## 第七章 像高手一样玩转数据
本章将学到许多操作数据的方法,它们大多与下面这两种内置的Python数据类型有关.
* 字符串
        Unicode字符组成的序列,用于存储文本数据
* 字节和字节数组
        8比特整数组成的序列,用于存储二进制数据

#### 文本字符串

* Python3中的字符串是Unicode字符串而不是字节数组 这是与Python2相比最大的差别

Python中的unicodedata模块提供了下面两个方向的转换函数:
* lookup()--接受不区分大小写的标准名称,返回一个Unicode字符
* name()--接受一个Unicode字符,返回一个大写形式的名称

In [1]:
def unicode_test(value):
    import unicodedata
    name = unicodedata.name(value)
    value2 = unicodedata.lookup(name)
    print('value = {},name = {},value2 = {}'.format(value,name,value2))

In [2]:
unicode_test('A')

value = A,name = LATIN CAPITAL LETTER A,value2 = A


In [3]:
unicode_test('$')

value = $,name = DOLLAR SIGN,value2 = $


In [4]:
unicode_test('\u00a2')

value = ¢,name = CENT SIGN,value2 = ¢


In [5]:
unicode_test('\u3fea')

value = 㿪,name = CJK UNIFIED IDEOGRAPH-3FEA,value2 = 㿪


In [6]:
unicode_test('\u2603')

value = ☃,name = SNOWMAN,value2 = ☃


* 使用UTF-8编码和解码
    * 将字符串编码为字节
    * 将字节解码为字符串

* 编码

In [8]:
snowman = '\u2603'
len(snowman)

1

In [10]:
ds = snowman.encode('utf-8')
ds

b'\xe2\x98\x83'

In [11]:
len(ds)

3

In [13]:
ds = snowman.encode('ascii')

UnicodeEncodeError: 'ascii' codec can't encode character '\u2603' in position 0: ordinal not in range(128)

In [14]:
ds = snowman.encode('ascii','ignore')
ds

b''

In [15]:
# replace会将无法编码的字符替换为?
ds = snowman.encode('ascii','replace')
ds

b'?'

In [16]:
# backslashreplace则会创建一个和unicode-escape类似的Unicode字符串:
ds = snowman.encode('ascii','backslashreplace')
ds

b'\\u2603'

In [17]:
# 下面的代码可以用于创建网页中使用的字符实体串
ds = snowman.encode('ascii','xmlcharrefreplace')
ds

b'&#9731;'

* 解码

In [18]:
place = 'caf\u00e9'
place

'café'

In [19]:
type(place)

str

In [20]:
place_bytes = place.encode('utf-8')
place_bytes

b'caf\xc3\xa9'

In [21]:
place2 = place_bytes.decode('utf-8')
place2

'café'

#### 格式化

* 使用%旧式格式化
![转换类型](http://owz0zbwsq.bkt.clouddn.com/%E8%BD%AC%E6%8D%A2%E7%B1%BB%E5%9E%8B.png)

In [24]:
'%-10s' % 'hello'

'hello     '

* 使用{}和format的新式格式化

In [25]:
'{}-{}-{}'.format(2017,11,2)

'2017-11-2'

In [27]:
'{2},{0},{1}'.format('zero','one','two')

'two,zero,one'

In [29]:
d = {'s':'nice','f':1.23,'n':55}
"{0[n]}|{0[f]}|{0[s]}-{1}".format(d,'other')

'55|1.23|nice-other'

In [31]:
'{0:>10d}{1:<10f}{2:>20s}'.format(12,3.14,'chinese')

'        123.140000               chinese'

#### 使用正则表达式匹配

In [35]:
import re

result = re.match('You','Young Frankenstein')
result

<_sre.SRE_Match object; span=(0, 3), match='You'>

* 对于复杂的匹配,可以先对模式进行1编译以加快匹配速度

In [36]:
youpattern = re.compile('You')
result = youpattern.match('Young Frankenstein')
result

<_sre.SRE_Match object; span=(0, 3), match='You'>

match()并不是比较source和pattern的唯一办法
* search()会返回第一次成功匹配,如果存在的话
* findall()会返回所有不重叠的匹配
* split()会根据pattern将source切分成若干段,返回这些片段组成的列表
* sub()还需要一个额外的参数replacement,它会把source中所有匹配的pattern改成replacement

模式:特殊的字符
![特殊的字符](http://owz0zbwsq.bkt.clouddn.com/%E7%89%B9%E6%AE%8A%E5%AD%97%E7%AC%A6.png)
模式:使用标识符
![模式标识符](http://owz0zbwsq.bkt.clouddn.com/%E6%A0%87%E8%AF%86%E7%AC%A61.png)
![模式标识符](http://owz0zbwsq.bkt.clouddn.com/%E6%A0%87%E8%AF%86%E7%AC%A62.png)

#### 二进制数据

* 字节是不可变的,像字节数据组成的元组
* 字节数组是可变的,像字节数据组成的列表

In [37]:
blist = [1,2,3,255]

the_bytes = bytes(blist)
the_bytes

b'\x01\x02\x03\xff'

In [38]:
the_byte_array = bytearray(blist)
the_byte_array

bytearray(b'\x01\x02\x03\xff')

In [40]:
the_bytes[1] = 2

TypeError: 'bytes' object does not support item assignment

In [42]:
the_byte_array[1] = 3
the_byte_array

bytearray(b'\x01\x03\x03\xff')

* 使用struct转换二进制数据

In [43]:
import struct

valid_png_header = b'\x89PNG\r\n\x1a\n'
data = b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR' + \
    b'\x00\x00\x00\x9a\x00\x00\x00\x8d\x08\x02\x00\x00\x00\xc0'
    
if data[:8] == valid_png_header:
    width, height = struct.unpack('>LL', data[16:24])
    print('Valid PNG, width', width, 'height', height)
else:
    print('Not a valid PNG')

Valid PNG, width 154 height 141
